In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample, shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
training_data = '../../data/training_set_all_features_no_onehot_scaling.csv'
# training_data = '../../data/training_set_features.csv'

training_labels = '../../data/training_set_labels.csv'

test_data = '../../data/test_set_all_features_no_onehot_scaling.csv'
# test_data = '../../data/test_set_features.csv'

test_labels = '../../data/test_set_labels.csv'

training_data = pd.read_csv(training_data)
training_labels = pd.read_csv(training_labels)
test_data = pd.read_csv(test_data)
test_labels = pd.read_csv(test_labels)

In [3]:
drop = ['QSTLANG', '_FRT16', '_FRTRESP', '_FRUITEX', '_MISFRTN', '_MISVEGN', 'DISPCODE']
training_data.drop(drop, inplace=True, axis=1)
test_data.drop(drop, inplace=True, axis=1)

In [4]:
training_data

,GRENDAY_,MENTHLTH,ORNGDAY_,HTIN4,WTKG3,FC60_,CHILDREN,MAXVO2_,PHYSHLTH,_VEGESUM,_DRNKWEK,STRFREQ_,VEGEDA1_,VEGETAB1,STRENGTH,_BMI5,DROCDY3_,BEANDAY_,ALCDAY5,FRUIT1,HAVARTH3,HLTHPLN1,MARITAL,MEDCOST,PERSDOC2,CHCCOPD1,_INCOMG,TOLDHI2,DIFFWALK,_LMTSCL1,_RFCHOL,QLACTLM2,PNEUVAC3,CVDSTRK3,DIABETE3,_AGE65YR,_RFHYPE5,_RFHLTH,SEX,_DRDXAR1,BPHIGH4,_AGE_G,_AGE80,GENHLTH,_HCVU651,EMPLOY1,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_RFBING5,_RFBMI5,_RFDRHV5,_RFSEAT2,_RFSEAT3,_RFSMOK3,_SMOKER3,_TOTINDA,_VEGLT1,RENTHOM1,SMOKE100,USEEQUIP,USENOW3,VETERAN3,_AIDTST3,_ASTHMS1,_BMI5CAT,_CHISPNC,_CHLDCNT,_CHOLCHK,_EDUCAG,_FRTLT1,_HISPANC,_LMTACT1,_LMTWRK1,_LTASTH1,_MRACE1,_PA150R2,_PA30021,_PA300R2,_PACAT1,_PAINDX1,_PAREC1,_PASTAE1,_PASTRNG,_PRACE1,ADDEPEV2,ASTHMA3,BLIND,BLOODCHO,CHCKIDNY,CHCOCNCR,CHCSCNCR,CHECKUP1,CHOLCHK,DECIDE,DIFFALON,DIFFDRES,DRNKANY5,EDUCA,EXERANY2,FLUSHOT6
0,14.0,5.0,17.0,66.0,5897.0,350.0,0.0,2040.0,20.0,131.0,5.397605e-79,5.397605e-79,100.0,101.0,13.0,2098.0,5.397605e-79,5.397605e-79,0.00,30.00,1.0,1.0,5.0,0.0,2.0,0.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,6.0,72.0,5.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,3.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0
1,71.0,0.0,57.0,72.0,8165.0,623.0,1.0,3635.0,0.0,171.0,2.330000e+02,7.000000e+03,29.0,202.0,30.0,2441.0,1.700000e+01,1.400000e+01,5.00,30.00,0.0,1.0,2.0,0.0,1.0,0.0,5.0,2.0,0.0,4.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,3.0,43.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,4.0,1.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,6.0,1.0,0.0
2,13.0,0.0,7.0,62.0,9072.0,525.0,0.0,3061.0,0.0,40.0,5.397605e-79,1.000000e+03,20.0,306.0,4.0,3658.0,5.397605e-79,5.397605e-79,0.00,1.00,1.0,1.0,6.0,0.0,3.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,1.0,3.0,4.0,47.0,3.0,1.0,8.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,4.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,0.0,4.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0
3,43.0,0.0,10.0,74.0,12247.0,482.0,0.0,2810.0,0.0,182.0,5.397605e-79,9.330000e+02,100.0,101.0,4.0,3467.0,5.397605e-79,2.900000e+01,0.00,30.00,0.0,1.0,1.0,0.0,2.0,0.0,5.0,2.0,1.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,58.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,3.0,4.0,2.0,1.0,1.0,3.0,1.0,0.0,3.0,3.0,0.0,1.0,2.0,2.0,2.0,3.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0
4,43.0,0.0,43.0,67.0,6577.0,334.0,0.0,1951.0,0.0,172.0,5.397605e-79,5.397605e-79,43.0,203.0,13.0,2271.0,5.397605e-79,4.300000e+01,0.00,60.00,0.0,1.0,1.0,0.0,1.0,0.0,4.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,6.0,77.0,3.0,1.0,7.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,1.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,3.0,3.0,0.0,2.0,3.0,2.0,3.0,4.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54081,29.0,0.0,43.0,63.0,6350.0,392.0,0.0,2284.0,1.0,143.0,2.000000e+02,5.397605e-79,57.0,204.0,13.0,2480.0,2.900000e+01,1.400000e+01,8.66,4.33,0.0,1.0,1.0,0.0,1.0,0.0,5.0,1.0,0.0,4.0,1.0,1.0,1.0,0.0,3.0,2.0,1.0,1.0,0.0,0.0,1.0,6.0,68.0,3.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,1.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,3.0,0.0,0.0,3.0,3.0,0.0,1.0,3.0

In [16]:
print(training_data.shape)
print(training_labels.shape)
print(test_data.shape)
print(test_labels.shape)

(54086, 244)
(54086, 1)
(23180, 244)
(23180, 1)


In [6]:
# categorical: missing values will be replaced with the mode.
categorical = [
    "HAVARTH3",
    "HLTHPLN1",
    "MARITAL",
    "MEDCOST",
    "PERSDOC2",
    "CHCCOPD1",
    "_INCOMG",
    "TOLDHI2",
    "DIFFWALK",
    "_LMTSCL1",
    "_RFCHOL",
    "QLACTLM2",
    "PNEUVAC3",
    "CVDSTRK3",
    "DIABETE3",
    "_AGE65YR",
    "_RFHYPE5",
    "_RFHLTH",
    "SEX",
    "_DRDXAR1",
    "BPHIGH4",
    "_AGE_G",
    "_AGE80",
    "GENHLTH",
    "_HCVU651",
    "EMPLOY1",
    "_RACE",
    "_RACEG21",
    "_RACEGR3",
    "_RACE_G1",
    "_RFBING5",
    "_RFBMI5",
    "_RFDRHV5",
    "_RFSEAT2",
    "_RFSEAT3",
    "_RFSMOK3",
    "_SMOKER3",
    "_TOTINDA",
    "_VEGLT1",
    "RENTHOM1",
    "SMOKE100",
    "USEEQUIP",
    "USENOW3",
    "VETERAN3",
    "_AIDTST3",
    "_ASTHMS1",
    "_BMI5CAT",
    "_CHISPNC",
    "_CHLDCNT",
    "_CHOLCHK",
    "_EDUCAG",
    "_FRTLT1",
    "_HISPANC",
    "_LMTACT1",
    "_LMTWRK1",
    "_LTASTH1",
    "_MRACE1",
    "_PA150R2",
    "_PA30021",
    "_PA300R2",
    "_PACAT1",
    "_PAINDX1",
    "_PAREC1",
    "_PASTAE1",
    "_PASTRNG",
    "_PRACE1",
    "ADDEPEV2",
    "ASTHMA3",
    "BLIND",
    "BLOODCHO",
    "CHCKIDNY",
    "CHCOCNCR",
    "CHCSCNCR",
    "CHECKUP1",
    "CHOLCHK",
    "DECIDE",
    "DIFFALON",
    "DIFFDRES",
    "DRNKANY5",
    "EDUCA",
    "EXERANY2",
    "FLUSHOT6",
]
# continuous: missing values will be replaced with the median. These need to be scaled.
continuous = [
    "GRENDAY_",
    "MENTHLTH",
    "ORNGDAY_",
    "HTIN4",
    "WTKG3",
    "FC60_",
    "CHILDREN",
    "MAXVO2_",
    "PHYSHLTH",
    "_VEGESUM",
    "_DRNKWEK",
    "STRFREQ_",
    "VEGEDA1_",
    "VEGETAB1",
    "STRENGTH",
    "_BMI5",
    "DROCDY3_",
    "BEANDAY_",
    "ALCDAY5",
    "FRUIT1",
]

target = ["_MICHD"]

In [ ]:
training_data[categorical] = training_data[categorical].round()
test_data[categorical] = test_data[categorical].round()

training_data[continuous] = training_data[continuous].round(3)
test_data[continuous] = test_data[continuous].round(3)

In [ ]:
training_data

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

cat_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False, drop='first')),
    ]
)


num_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        # ("std_scaler", StandardScaler()),
    ]
)

full_pipeline = ColumnTransformer(
    [
        ("num", num_pipeline, continuous),
        ("cat", cat_pipeline, categorical),
    ]
)

In [8]:
full_pipeline.fit(training_data)
# Transform the training data and convert it back to a DataFrame with column names
training_data = pd.DataFrame(
    full_pipeline.transform(training_data),
    columns=continuous + full_pipeline.named_transformers_["cat"].get_feature_names_out(categorical).tolist(),
)
test_data = pd.DataFrame(
    full_pipeline.transform(test_data),
    columns=continuous + full_pipeline.named_transformers_["cat"].get_feature_names_out(categorical).tolist(),
)

/Users/marinus/miniconda3/envs/gpu_venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
training_data = pd.get_dummies(training_data, columns=categorical)
test_data = pd.get_dummies(test_data, columns=categorical)
test_data

In [17]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(training_data, training_labels.values.ravel())
importances = clf.feature_importances_

feature_list = list(training_data.columns)
feature_importance = pd.DataFrame({'Feature': feature_list, 'Importance': importances})
sorted_importance = feature_importance.sort_values('Importance', ascending=False)
sorted_importance

,Feature,Importance
3,HTIN4,0.014749
4,WTKG3,0.014538
15,_BMI5,0.014395
17,BEANDAY_,0.014321
13,VEGETAB1,0.014275
...,...,...
138,_RACE_6.0,0.000376
195,_MRACE1_5.0,0.000277
137,_RACE_5.0,0.000216
217,_PRACE1_7.0,0.000196


In [ ]:
important = list(sorted_importance[:50].Feature)
important

In [14]:
important = list(sorted_importance.Feature)

In [12]:
training_data

,GRENDAY_,MENTHLTH,ORNGDAY_,HTIN4,WTKG3,FC60_,CHILDREN,MAXVO2_,PHYSHLTH,_VEGESUM,_DRNKWEK,STRFREQ_,VEGEDA1_,VEGETAB1,STRENGTH,_BMI5,DROCDY3_,BEANDAY_,ALCDAY5,FRUIT1,HAVARTH3_1.0,HLTHPLN1_1.0,MARITAL_2.0,MARITAL_3.0,MARITAL_4.0,MARITAL_5.0,MARITAL_6.0,MEDCOST_1.0,PERSDOC2_2.0,PERSDOC2_3.0,CHCCOPD1_1.0,_INCOMG_2.0,_INCOMG_3.0,_INCOMG_4.0,_INCOMG_5.0,TOLDHI2_2.0,DIFFWALK_1.0,_LMTSCL1_2.0,_LMTSCL1_3.0,_LMTSCL1_4.0,_RFCHOL_1.0,QLACTLM2_1.0,PNEUVAC3_1.0,CVDSTRK3_1.0,DIABETE3_2.0,DIABETE3_3.0,DIABETE3_4.0,_AGE65YR_2.0,_RFHYPE5_1.0,_RFHLTH_2.0,SEX_1.0,_DRDXAR1_1.0,BPHIGH4_2.0,BPHIGH4_3.0,BPHIGH4_4.0,_AGE_G_2.0,_AGE_G_3.0,_AGE_G_4.0,_AGE_G_5.0,_AGE_G_6.0,_AGE80_19.0,_AGE80_20.0,_AGE80_21.0,_AGE80_22.0,_AGE80_23.0,_AGE80_24.0,_AGE80_25.0,_AGE80_26.0,_AGE80_27.0,_AGE80_28.0,_AGE80_29.0,_AGE80_30.0,_AGE80_31.0,_AGE80_32.0,_AGE80_33.0,_AGE80_34.0,_AGE80_35.0,_AGE80_36.0,_AGE80_37.0,_AGE80_38.0,_AGE80_39.0,_AGE80_40.0,_AGE80_41.0,_AGE80_42.0,_AGE80_43.0,_AGE80_44.0,_AGE80_45.0,_AGE80_46.0,_AGE80_47.0,_AGE80_48.0,_AGE80_49.0,_AGE80_50.0,_AGE80_51.0,_AGE80_52.0,_AGE80_53.0,_AGE80_54.0,_AGE80_55.0,_AGE80_56.0,_AGE80_57.0,_AGE80_58.0,_AGE80_59.0,_AGE80_60.0,_AGE80_61.0,_AGE80_62.0,_AGE80_63.0,_AGE80_64.0,_AGE80_65.0,_AGE80_66.0,_AGE80_67.0,_AGE80_68.0,_AGE80_69.0,_AGE80_70.0,_AGE80_71.0,_AGE80_72.0,_AGE80_73.0,_AGE80_74.0,_AGE80_75.0,_AGE80_76.0,_AGE80_77.0,_AGE80_78.0,_AGE80_79.0,_AGE80_80.0,GENHLTH_2.0,GENHLTH_3.0,GENHLTH_4.0,GENHLTH_5.0,_HCVU651_1.0,EMPLOY1_2.0,EMPLOY1_3.0,EMPLOY1_4.0,EMPLOY1_5.0,EMPLOY1_6.0,EMPLOY1_7.0,EMPLOY1_8.0,_RACE_2.0,_RACE_3.0,_RACE_4.0,_RACE_5.0,_RACE_6.0,_RACE_7.0,_RACE_8.0,_RACEG21_2.0,_RACEGR3_2.0,_RACEGR3_3.0,_RACEGR3_4.0,_RACEGR3_5.0,_RACE_G1_2.0,_RACE_G1_3.0,_RACE_G1_4.0,_RACE_G1_5.0,_RFBING5_2.0,_RFBMI5_2.0,_RFDRHV5_2.0,_RFSEAT2_2.0,_RFSEAT3_2.0,_RFSMOK3_2.0,_SMOKER3_2.0,_SMOKER3_3.0,_SMOKER3_4.0,_TOTINDA_2.0,_VEGLT1_2.0,RENTHOM1_2.0,RENTHOM1_3.0,SMOKE100_2.0,USEEQUIP_2.0,USENOW3_2.0,USENOW3_3.0,VETERAN3_2.0,_AIDTST3_2.0,_ASTHMS1_2.0,_ASTHMS1_3.0,_BMI5CAT_2.0,_BMI5CAT_3.0,_BMI5CAT_4.0,_CHISPNC_2.0,_CHLDCNT_2.0,_CHLDCNT_3.0,_CHLDCNT_4.0,_CHLDCNT_5.0,_CHLDCNT_6.0,_CHOLCHK_2.0,_CHOLCHK_3.0,_EDUCAG_2.0,_EDUCAG_3.0,_EDUCAG_4.0,_FRTLT1_1.0,_HISPANC_1.0,_LMTACT1_2.0,_LMTACT1_3.0,_LMTWRK1_2.0,_LMTWRK1_3.0,_LTASTH1_1.0,_MRACE1_2.0,_MRACE1_3.0,_MRACE1_4.0,_MRACE1_5.0,_MRACE1_6.0,_MRACE1_7.0,_PA150R2_2.0,_PA150R2_3.0,_PA30021_2.0,_PA300R2_2.0,_PA300R2_3.0,_PACAT1_2.0,_PACAT1_3.0,_PACAT1_4.0,_PAINDX1_1.0,_PAREC1_2.0,_PAREC1_3.0,_PAREC1_4.0,_PASTAE1_1.0,_PASTRNG_1.0,_PRACE1_2.0,_PRACE1_3.0,_PRACE1_4.0,_PRACE1_5.0,_PRACE1_6.0,_PRACE1_7.0,_PRACE1_8.0,ADDEPEV2_1.0,ASTHMA3_1.0,BLIND_1.0,BLOODCHO_1.0,CHCKIDNY_1.0,CHCOCNCR_1.0,CHCSCNCR_1.0,CHECKUP1_2.0,CHECKUP1_3.0,CHECKUP1_4.0,CHECKUP1_8.0,CHOLCHK_2.0,CHOLCHK_3.0,CHOLCHK_4.0,DECIDE_1.0,DIFFALON_1.0,DIFFDRES_1.0,DRNKANY5_1.0,EDUCA_2.0,EDUCA_3.0,EDUCA_4.0,EDUCA_5.0,EDUCA_6.0,EXERANY2_1.0,FLUSHOT6_1.0
0,14.0,5.0,17.0,66.0,5897.0,350.0,0.0,2040.0,20.0,131.0,5.397605e-79,5.397605e-79,100.0,101.0,13.0,2098.0,5.397605e-79,5.397605e-79,0.00,30.00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1,71.0,0.0,57.0,72.0,8165.0,623.0,1.0,3635.0,0.0,171.0,2.330

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

def evaluate_model(model, name, X_train, X_test, y_train, y_test):
    if name == "LogisticRegression":
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        model.fit(X_train_scaled, y_train)
        X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns=X_train.columns)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        print(f"Training accuracy: {f1_score(y_train, y_train_pred)}")
        y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    print(f"{name}:", f1)
    return f1

models = {
    # "CatBoostClassifier": CatBoostClassifier(
    #     silent=True, 
    #     random_seed=42, 
    #     iterations=1000, 
    #     depth=4, 
    #     # learning_rate=0.1,
    #     # cat_features=categorical
    # ),
    # "AdaBoostClassifier": AdaBoostClassifier(
    #     n_estimators=200, 
    #     # learning_rate=1.0
    # ),
    "RandomForestClassifier": RandomForestClassifier(
        n_estimators=1000,
        max_depth=6,
        # max_features='sqrt', 
        # min_samples_split=10, 
        # min_samples_leaf=4, 
        random_state=42
    ),
    # "GradientBoostingClassifier": GradientBoostingClassifier(
    #     n_estimators=150, 
    #     # learning_rate=0.1, 
    #     max_depth=4
    # ),
    # "LGBMClassifier": LGBMClassifier(
    #     num_leaves=31, 
    #     max_depth=-1, 
    #     learning_rate=0.1, 
    #     verbose=-1
    # ),
    # "LogisticRegression": LogisticRegression(max_iter=1000),
    # "SVC": SVC(),
    # "KNeighborsClassifier": KNeighborsClassifier(),
    # "GaussianNB": GaussianNB(),
    # "MLPClassifier": MLPClassifier(hidden_layer_sizes=(90), random_state=42),
    # "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    # "ExtraTreesClassifier": ExtraTreesClassifier(n_estimators=100)
}

results = {name: evaluate_model(model, name, training_data[important], test_data[important], training_labels.values.ravel(), test_labels.values.ravel()) for name, model in models.items()}
sorted_results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))
sorted_results

Training accuracy: 0.562911132526082
RandomForestClassifier: 0.43831640058055155


{'RandomForestClassifier': 0.43831640058055155}